In [2]:
#Las cosas importantes porque son importadas

import cv2  
import math 

from ultralytics import YOLO

from collections import defaultdict
import numpy as np

import cv2
import pytesseract
from pytesseract import Output

model = YOLO('yolov11-plates\weights\yolov11-plates-best.pt') #Contenedores

filename = "Parking.mp4"
results = model(filename, show=True)

cv2.destroyAllWindows()



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/771) c:\Users\JJ\Desktop\VC\VC_P4\Parking.mp4: 384x640 (no detections), 27.7ms
video 1/1 (frame 2/771) c:\Users\JJ\Desktop\VC\VC_P4\Parking.mp4: 384x640 (no detections), 14.3ms
video 1/1 (frame 3/771) c:\Users\JJ\Desktop\VC\VC_P4\Parking.mp4: 384x640 (no detections), 15.2ms
video 1/1 (frame 4/771) c:\Users\JJ\Desktop\VC\VC_P4\Parking.mp4: 384x640 (no detections), 13.3ms
video 1/1 (frame 5/771) c:\Users\JJ\Desktop\VC\VC_P4\Parking.mp4: 

In [15]:
from ultralytics import YOLO
import cv2
import math
import pytesseract
import easyocr


model_person_vehicle = YOLO("yolo11n.pt")

model_license_plate = YOLO("yolov11-plates\weights\yolov11-plates-last.pt")

classNames = ["person", "", "car"]

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'
custom_config = r'--oem 3 --psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

reader = easyocr.Reader(['en'])

cap = cv2.VideoCapture("CochesC.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results_person_vehicle = model_person_vehicle(frame)
    
    # Iterar sobre cada detección
    for result in results_person_vehicle:
        boxes = result.boxes
        for box in boxes:
            cls = int(box.cls[0])  # Clase de la detección
            if cls == 0 or cls == 2:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
            
                            # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confianza --->",confidence)

                # Clase
                # cls = int(box.cls[0])
                print("Clase -->", classNames[cls])

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0
                
                if cls == 2:
                    # Detección de matrículas
                    results_license_plate = model_license_plate(frame[y1:y2, x1:x2])

                    for result in results_license_plate:
                        boxes = result.boxes
                        for box in boxes:
                            xp1, yp1, xp2, yp2 = map(int, box.xyxy[0])

                            cv2.rectangle(frame, (xp1+x1, yp1+y1), (xp2+x1, yp2+y1), (0, 255, 0), 3)

                            # Detección de matrículas
                            license_plate = frame[yp1+y1:yp2+y1, xp1+x1:xp2+x1]

                            # Canny
                            gray_plate = cv2.cvtColor(license_plate, cv2.COLOR_BGR2GRAY)
                            gaussian_plate = cv2.GaussianBlur(gray_plate, (3, 3), 0)
                            _, binary_plate = cv2.threshold(gaussian_plate, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

                            contornos, hierarchy = cv2.findContours(binary_plate, 
                                cv2.RETR_EXTERNAL , 
                                cv2.CHAIN_APPROX_SIMPLE)

                            #Recorre los contornos externos
                            for c in contornos:
                                area = cv2.contourArea(c)
                                if area > 10:
                                    # dibuja los contornos teniendo en cuenta el despazamiento de la matrícula
                                    cv2.drawContours(license_plate, [c], -1, (255,0,0), -1)

                            # Pasar la imagen procesada a pytesseract
                            license_plate_text = pytesseract.image_to_string(gaussian_plate, config=custom_config)
                            print("Matrícula detectada:", license_plate_text)

                            # Lectura con EasyOCR
                            result = reader.readtext(gaussian_plate)
                            print("Matrícula EASYOCR:", result)


                            # Dibuja el contenedor y matrícula
                            cv2.putText(frame, license_plate_text, [xp1+x1, yp1+y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                            print("Matrícula -->", license_plate_text)



                        
    # Mostrar el fotograma procesado (opcional)
    cv2.imshow("Detección de vehículos y matrículas", frame)
    
    # Detenemos pulsado ESC
    if cv2.waitKey(20) == 27:
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 7 cars, 15.2ms
Speed: 1.6ms preprocess, 15.2ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.9
Clase --> car

0: 352x640 2 plates, 13.2ms
Speed: 1.0ms preprocess, 13.2ms inference, 3.0ms postprocess per image at shape (1, 3, 352, 640)
Matrícula detectada: 
Matrícula EASYOCR: []
Matrícula --> 
Matrícula detectada: 
Matrícula EASYOCR: []
Matrícula --> 
Confianza ---> 0.89
Clase --> car

0: 512x640 1 plate, 14.1ms
Speed: 1.0ms preprocess, 14.1ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)
Matrícula detectada: 
Matrícula EASYOCR: []
Matrícula --> 
Confianza ---> 0.86
Clase --> car

0: 416x640 1 plate, 12.8ms
Speed: 1.0ms preprocess, 12.8ms inference, 3.6ms postprocess per image at shape (1, 3, 416, 640)
Matrícula detectada: 
Matrícula EASYOCR: []
Matrícula --> 
Confianza ---> 0.75
Clase --> car

0: 544x640 1 plate, 13.1ms
Speed: 1.7ms preprocess, 13.1ms inference, 2.0ms postprocess per image at shape (1, 3, 544, 640)
Matr

In [35]:
from ultralytics import YOLO
import cv2
import math
import pytesseract
import easyocr

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'
reader = easyocr.Reader(['en'])

custom_config = r'--oem 3 --psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

img = cv2.imread('plate-example2.jpg')

if img is not None:
    #Convierte a RGB antes de procesar
    gray_plate = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(gray_plate, (3, 3), 0)

    _, thresh_plate = cv2.threshold(blur, 90, 190, cv2.THRESH_BINARY_INV)

    contornos, hierarchy = cv2.findContours(thresh_plate, 
        cv2.RETR_EXTERNAL , 
        cv2.CHAIN_APPROX_SIMPLE)
    
    for contorno in contornos:
        area = cv2.contourArea(contorno)
        if area > 10:
            cv2.drawContours(thresh_plate, [contorno], -1, (255,0,0), -1)
    
    #muestra la imagen procesada
    cv2.imshow("Imagen procesada", thresh_plate)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    #Texto localizado
    teseracto = pytesseract.image_to_string(thresh_plate, config=custom_config)
    print("Texto detectado por Tesseract:", teseracto)

else:
    print('Error de imagen')

Texto detectado por Tesseract: S



In [17]:
# Import required packages
import cv2
import pytesseract

# Mention the installed location of Tesseract-OCR in your system
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

# Read image from which text needs to be extracted
img = cv2.imread("plate-example1.png")

# Preprocessing the image starts

# Convert the image to gray scale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Performing OTSU threshold
ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

# Specify structure shape and kernel size. 
# Kernel size increases or decreases the area 
# of the rectangle to be detected.
# A smaller value like (10, 10) will detect 
# each word instead of a sentence.
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))

# Applying dilation on the threshold image
dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)

# Finding contours
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, 
                                                 cv2.CHAIN_APPROX_NONE)

# Creating a copy of image
im2 = img.copy()

# A text file is created and flushed
file = open("recognized.txt", "w+")
file.write("")
file.close()

# Looping through the identified contours
# Then rectangular part is cropped and passed on
# to pytesseract for extracting text from it
# Extracted text is then written into the text file
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    
    # Drawing a rectangle on copied image
    rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Cropping the text block for giving input to OCR
    cropped = im2[y:y + h, x:x + w]
    
    # Open the file in append mode
    file = open("recognized.txt", "a")
    
    # Apply OCR on the cropped image
    text = pytesseract.image_to_string(cropped)
    
    # Appending the text into file
    file.write(text)
    file.write("\n")
    
    # Close the file
    file.close()


cv2.imshow('Image', im2)
if cv2.waitKey(20) == 27:
    cv2.destroyAllWindows()
# This code is modified by Susobhan Akhuli